In [6]:
import pandas as pd
import pyprind 
import os
import sys

In [7]:
basepath = "aclImdb"
labels = {'pos':1,'neg':0}
pbar = pyprind.ProgBar(50000,  stream=sys.stdout)
data = [] 
for s in ("test","train"):
    for l in ("pos","neg"):
        path = os.path.join(basepath,s,l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
                txt = infile.read()
                data.append([txt,labels[l]])
            pbar.update()

df = pd.DataFrame(data,columns=["review","sentiment"])

0% [#                             ] 100% | ETA: 00:00:01

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [8]:
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [21]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv',index=False,encoding='utf-8')

In [20]:
df.loc[:,'sentiment'].sum()

np.int64(25000)

In [24]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>',"",text)
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)",text)
    text = re.sub('[\W]+'," ",text.lower())+"".join(emoticons).replace('-','')
    
    return text

<>:5: SyntaxWarning: invalid escape sequence '\)'
<>:6: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\)'
<>:6: SyntaxWarning: invalid escape sequence '\W'
/tmp/ipykernel_5628/64227290.py:5: SyntaxWarning: invalid escape sequence '\)'
  emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)",text)
/tmp/ipykernel_5628/64227290.py:6: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('[\W]+'," ",text.lower())+"".join(emoticons).replace('-','')


In [26]:
df["review"] = df["review"].apply(preprocessor)

In [36]:
def tokenizer(text):
    return text.split()


In [32]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nishantk30/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [33]:
X_train = df.loc[:40000,'review'].values
y_train = df.loc[:40000,'sentiment'].values
X_test = df.loc[40000:, 'review'].values
y_test = df.loc[40000:,'sentiment'].values

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)

small_param_grid = [
    {
        'vect__ngram_range':[(1,1)],
        'vect__stop_words':[None],
        'vect__tokenizer':[tokenizer,tokenizer_porter],
        'clf__penalty':['l2'],
        'clf__C':[1.0,10.0]
    },
    {
        'vect__ngram_range':[(1,1)],
        'vect__stop_words':[stop,None],
        'vect__tokenizer':[tokenizer],
        'vect__use_idf':[False],
        'vect__norm':[None],
        'clf__penalty':['l2'],
        'clf__C':[1.0,10.0] 
    },
]

lr_tfidf = Pipeline([
    ('vect',tfidf),
    ('clf',LogisticRegression(solver='liblinear'))
])

gs_lr_tfdif = GridSearchCV(lr_tfidf,small_param_grid,scoring='accuracy',cv=5,verbose=2,n_jobs=-1)

gs_lr_tfdif.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f81809982c0>; total time=   4.7s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f5e5e1f02c0>; total time=   4.9s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7fa08b7542c0>; total time=   4.8s
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f61257f02c0>; total time=   5.1s


/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f6c2aef42c0>; total time=   5.8s


/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f72c9de6340>; total time=   6.7s


...

[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f40d6b0e340>, vect__use_idf=False; total time=  24.9s


/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 

/home/nishantk30/miniconda3/envs/torchenv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7fb4546c3600>,
                                              <function tokenizer_porter at 0x7fb4cb5522a0...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x7fb4546c3600>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [42]:
print(f"Best parameter settings : {gs_lr_tfdif.best_params_}")
print(f"CV accuracy : {gs_lr_tfdif.best_score_}")
clf = gs_lr_tfdif.best_estimator_
print(f"Test Accuracy : {clf.score(X_test,y_test):3f}")

Best parameter settings : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7fb4546c3600>}
CV accuracy : 0.8956423392757138
Test Accuracy : 0.899097
